In [1]:
import sys
from pathlib import Path
# code_path = Path('../../').resolve()
# sys.path.append(str(code_path))

In [ ]:
%load_ext autoreload
%autoreload 2
import _path
import powerlaw
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import scipy.stats as stats
import queue
import seaborn
import random
from scipy.optimize import leastsq
from rw_data_processing import *
from Data_synthesize import *
from transition_probability_estimation import *
from plot_results import *
from firefly_optimizer import *
plt.style.use("../rw_visualization.mplstyle")


In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Color
current_palette = seaborn.color_palette()
current_palette

# 1. Cheng2020

## 1.0 Lode main simulation restuls

In [ ]:
data_path = Path('../synthetic_data_results_cheng2020')
demographic_data_list, social_data_list, course_of_disease_data_list, contact_data_list, case_edge_list_list = \
    load_synthetic_data(data_path, monte_carlo_number=100)

In [14]:
demographic_data_all = np.concatenate(demographic_data_list, axis=0)
social_data_all = np.concatenate(social_data_list, axis=0)
contact_data_all = np.concatenate(contact_data_list, axis=0)
course_of_disease_data_all = np.concatenate(course_of_disease_data_list, axis=0)
case_edge_list_all = np.concatenate(case_edge_list_list, axis=0)


## 1.1 Cheng2020 Figure 2

In [ ]:
household_lower_bound = np.array([1.6, 1.4, 3, 0, 0, 0])/100
household_upper_bound = np.array([9.8, 16.8, 56.4, 49, 65.7, 100])/100
household_range = household_upper_bound-household_lower_bound
household_weight = 1/household_range
norm_household_weight = household_weight/max(household_weight)
print('household_weight: ', norm_household_weight)

In [ ]:
health_care_lower_bound = np.array([0.2, 0.7, 0.5, 0, 0, 0])/100
health_care_upper_bound = np.array([3, 5.7, 13.5, 18.4, 3.3, 2.6])/100
health_care_range = health_care_upper_bound-health_care_lower_bound
health_care_weight = 1/health_care_range
norm_health_care_weight = health_care_weight/max(health_care_weight)
print('health_care_weight: ', norm_health_care_weight)

In [ ]:
other_lower_bound = np.array([0, 0, 0.1, 0, 0, 0])/100
other_upper_bound = np.array([1, 0.6, 3.3, 4.2, 1.1, 3.3])/100
other_range = other_upper_bound-other_lower_bound
other_weight = 1/other_range
norm_other_weight = other_weight/max(other_weight)
print('other_weight: ', norm_other_weight)

In [18]:
total_household_contact_numbers = []
for i in range(100):
    tmp = []
    for contact_data in contact_data_all[i*100:(i+1)*100]:
        tmp.append(len(contact_data['household_contacts_matrix']))
    total_household_contact_numbers.append(sum(tmp))

In [ ]:
# Household
layer = 'Household'
ax1, ax2 = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100, layer=layer, save_fig=False)
ax1.set_ylim(0, 150)
ax1.set_ylabel('Household contacts')
ax1.set_xlabel("")
ax2.set_ylim(0, 65)
ax2.set_ylabel("")
# ax1.set_title('Synthetic data')
# plt.savefig('RW2022_synthesis_%s_contact_bar_chart.pdf' % layer)

ax1, ax2 = plot_cheng2020_bar_chart(layer='Household', save_fig=False)
ax1.set_ylim(0, 150)
# ax1.set_ylabel('Household contacts')
ax1.set_xlabel("")
ax1.set_ylabel("")
ax2.set_ylim(0, 65)
# ax1.set_title('Cheng2020')
plt.savefig('RW2022_cheng_%s_bar_chart.pdf' % layer)


In [ ]:
# Health care
layer = 'Health care'
ax1, ax2 = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100,
    layer=layer, save_fig=False)
ax1.set_ylim(0, 400)
ax1.set_ylabel('Health care contacts')
ax1.set_xlabel("")
ax2.set_ylim(0, 10)
ax2.set_ylabel("")
# ax1.set_title('Synthetic data')
plt.savefig('RW2022_synthesis_%s_contact_bar_chart.pdf' % layer.replace(' ', '_'))


ax1, ax2 = plot_cheng2020_bar_chart(layer='Health care', save_fig=False)
ax1.set_ylim(0, 400)
ax1.set_xlabel("")
ax2.set_ylim(0, 10)
# ax1.set_title('Cheng2020')
plt.savefig('RW2022_cheng_%s_bar_chart.pdf' % layer.replace(' ', '_'))


In [ ]:
# School
ax1, ax2 = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100,
    layer='School', save_fig=False)
ax1.set_ylabel('School contacs')
ax1.set_ylim([0, 100])


In [ ]:
# Workplace
ax1, ax2 = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=100, num_source_cases=100,
    layer='Workplace', save_fig=False)
ax1.set_ylabel('Workplace contacts')


In [ ]:
# All
layer = 'All'
ax1, ax2 = plot_cheng2020_fig2(
    course_of_disease_data_all, contact_data_all, monte_carlo_number=1000, num_source_cases=100,
    layer=layer, save_fig=False)
ax1.set_ylim(0, 1200)
ax2.set_ylim(0, 10)
ax1.set_ylabel('All contacts')
ax2.set_ylabel("")
# ax1.set_title('Synthetic data')
plt.savefig('RW2022_synthesis_%s_contact_bar_chart.pdf' % layer)

ax1, ax2 = plot_cheng2020_bar_chart(layer='All', save_fig=False)
ax1.set_ylim(0, 1200)
ax2.set_ylim(0, 10)
# ax1.set_title('Cheng2020')
plt.savefig('RW2022_cheng_%s_bar_chart.pdf' % layer)


## 1.2 Plot the detail contact date and infection data bar char

In [ ]:
contact_day_shift, exposure_day_array, daily_contact_array, infection_day_shift, daily_infection_array, daily_secondary_attack_rate = \
    generate_contact_day_vs_infection_day_array(course_of_disease_data_all[0:1], contact_data_all[0:1])
print(f'contact_day_shift: {contact_day_shift}')
print(f'exposure_day_array: {exposure_day_array}')
print(f'daily_contact_array: {daily_contact_array}')
print(f'infection_day_shift: {infection_day_shift}')
print(f'daily_infection_array: {daily_infection_array}')
print(f'daily_secondary_attack_rate: {daily_secondary_attack_rate}')

In [ ]:
save_fig = False
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=10, monte_carlo_number=100, num_source_cases=100, layer='Household', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=5, monte_carlo_number=100, num_source_cases=100, layer='School', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=5, monte_carlo_number=100, num_source_cases=100, layer='Workplace', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=5, monte_carlo_number=100, num_source_cases=100, layer='Health care', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=2, monte_carlo_number=100, num_source_cases=100, layer='Municipality', save_fig=save_fig)
plot_contact_day_vs_infection_day(course_of_disease_data_all, contact_data_all, attack_y_limit=2, monte_carlo_number=100,num_source_cases=100, layer='All', save_fig=save_fig)

## 1.3 Plot logistic contact p example

In [ ]:
def generate_contact_p(t, ps, ph, s, ts, tr):
    y = ph + (ph-ps) - (ph-ps)/(1+np.exp(-s*(t-ts))) - (ph-ps)/(1+np.exp(s*(t-tr)))

    return(y)

ps, ph, s, ts, tr = 0.4, 0.8, 2, 5, 15
t = np.arange(-10, 50, 0.1)
y = generate_contact_p(t, ps, ph, s, ts, tr)
plt.figure(figsize=(8, 2))
plt.plot(t, y)
plt.xlim([0, 20])
plt.ylim([0.3, 0.9])
plt.xticks(np.array([0, ts, tr]), ['$\mathrm{t_{I^S}}$', '$t_{PS}$', '$t_{PN}$'])
plt.yticks(np.array([ps, ph]), ['$p_S$', '$p_H$'])
plt.grid()
plt.savefig('RW2022_contact_p.pdf')

In [ ]:
def generate_logistic_contact_p(t, healthy_p, symptom_p, steepness, symptom_phase, width):
        # generate_logistic_contact_p: generate contact probability on day t
        #     healthy_p: Contact probability when healthy
        #     symptom_p: Contact probability when symptomatic
        #     steepness: Steepness of the logistic function
        #     symptom_phase: Phase relative to symptom-onset for symptomatic (days)
        #     width: Days different between symptom phase and normal phase
        normal_phase = symptom_phase + width
        logistic_p = healthy_p + (healthy_p-symptom_p) - (healthy_p-symptom_p)/(1+np.exp(-steepness*(
            t-symptom_phase))) - (healthy_p-symptom_p)/(1+np.exp(steepness*(t-normal_phase)))

        return (logistic_p)


contact_p, contact_previous_day_p, healthy_p, symptom_p = 0.1, 0.5, 0.1, 0.5
p = [contact_p, contact_previous_day_p, healthy_p, symptom_p]
steepness, symptom_phase, recover_phase = 10, 5, 2

t = np.arange(0, 50, 0.1)
logistic_p = generate_logistic_contact_p(t, healthy_p, symptom_p, steepness, symptom_phase, recover_phase)

plt.plot(t, logistic_p)
plt.grid()
plt.xlim([0, 20])

## 1.4 Plot contact number histogram

In [ ]:
contact_date_stack_matrix, index = generate_stack_contact_matrix(contact_data_all[0:100], layer='Household')
plt.hist(np.diff(index, prepend=0), bins=100)


In [31]:
household_effective_contact_array = np.array([])
school_effective_contact_array = np.array([])
workplace_effective_contact_array = np.array([])
health_care_effective_contact_array = np.array([])
municipality_effective_contact_array = np.array([])
for i in range(100):
    if np.nansum(contact_data_all[i]['household_effective_contacts_infection_time']) > 0:
        household_effective_contact_array = np.append(household_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['household_effective_contacts_infection_time']))))
    else:
        household_effective_contact_array = np.append(
            household_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['school_effective_contacts_infection_time']) > 0:
        school_effective_contact_array = np.append(school_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['school_effective_contacts_infection_time']))))
    else:
        school_effective_contact_array = np.append(
            school_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['workplace_effective_contacts_infection_time']) > 0:
        workplace_effective_contact_array = np.append(workplace_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['workplace_effective_contacts_infection_time']))))
    else:
        workplace_effective_contact_array = np.append(
            workplace_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['health_care_effective_contacts_infection_time']) > 0:
        health_care_effective_contact_array = np.append(health_care_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['health_care_effective_contacts_infection_time']))))
    else:
        health_care_effective_contact_array = np.append(
            health_care_effective_contact_array, 0)

    if np.nansum(contact_data_all[i]['municipality_effective_contacts_infection_time']) > 0:
        municipality_effective_contact_array = np.append(municipality_effective_contact_array, np.count_nonzero(
            ~np.isnan(np.array(contact_data_all[i]['municipality_effective_contacts_infection_time']))))
    else:
        municipality_effective_contact_array = np.append(
            municipality_effective_contact_array, 0)

all_effective_contact_array = household_effective_contact_array + school_effective_contact_array + \
    workplace_effective_contact_array + health_care_effective_contact_array + municipality_effective_contact_array


In [ ]:
plt.hist(all_effective_contact_array)